<a href="https://colab.research.google.com/github/arthursl12/POC1/blob/main/POC2_FD003_MLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [1]:
%pip install scikeras;
%pip install -U tensorflow-addons;

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import random
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import glob
import datetime

H:\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
H:\anaconda3\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
H:\anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [3]:
from sklearn.pipeline import Pipeline
from sklearn.compose import TransformedTargetRegressor

from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import MinMaxScaler, StandardScaler

from sklearn.base import BaseEstimator,RegressorMixin

In [4]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout

from tensorflow.keras.metrics import RootMeanSquaredError as RMSE

from keras.callbacks import LambdaCallback
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
from tensorflow.keras.optimizers.schedules import ExponentialDecay

import tensorflow_addons as tfa
from tensorflow_addons.metrics import RSquare as R2



from scikeras.wrappers import KerasRegressor

In [5]:
sns.set_palette('colorblind')

In [6]:
# Reproducibility
seed = 42
os.environ['PYTHONHASHSEED']=str(seed)
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)

In [7]:
tf.config.experimental.enable_op_determinism()

# Data Preparation

In [8]:
if 'google.colab' in str(get_ipython()):
    print('Running on CoLab')
    IN_COLAB = True
    folder="/content/CMaps/"

    # Dataset Download 
    os.system('git clone https://github.com/arthursl12/dataset_2')
    os.system('mv /content/dataset_2/CMaps /content/CMaps')
    os.system('mv /content/dataset_2/data_processing /content/data_processing')
    os.system('rm -rf dataset_2')
else:
    print('Not running on CoLab')
    IN_COLAB = False
    folder="CMaps/"
    %cd dataset_2/

Not running on CoLab
C:\Users\Arthur Lima\POC\dataset_2


In [9]:
from data_processing.processing import DatasetProcessing
from data_processing.training import HyperparameterSearch, reclipper_scorer
from data_processing.eval import Evaluation

In [10]:
proc = DatasetProcessing()

## Data Integration

The data are provided as a zip-compressed text file with 26 columns of numbers, separated by spaces. Each row is a snapshot of data taken during a single operational cycle, each column is a different variable. The columns correspond to:  

1) unit number   
2) time, in cycles  
3) operational setting 1  
4) operational setting 2  
5) operational setting 3    
6) sensor measurement 1    
7) sensor measurement 2  
...  
26) sensor measurement 20


There are 6 conditions (or combinations) which the 3 operational settings can take.  
Condition 1: Altitude = 0, Mach Number = 0, TRA = 100  
Condition 2: Altitude = 10, Mach Number = 0.25, TRA = 100  
Condition 3: Altitude = 20, Mach Number = 0.7 TRA = 100  
Condition 4: Altitude = 25, Mach Number = 0.62, TRA = 60  
Condition 5: Altitude = 35 Mach Number = 0.84, TRA = 100  
Condition 6: Altitude = 42, Mach Number = 0.84, TRA = 100  
  
There is slight variation in all these conditions so you may get numbers like 24.453 instead of 25 exactly.

FD001: Condition 1 only  
FD002: Mix of all the conditions  
FD003: Condition 1 only  
FD004: Mix of all conditions  


In [11]:
index_cols, settings_cols, sensors_cols, cols = proc.column_names()
train, test, y_test = proc.read_dataset(3,folder=folder)
train

,unit_number,time,op_1,op_2,op_3,s_0,s_1,s_2,s_3,s_4,...,s_11,s_12,s_13,s_14,s_15,s_16,s_17,s_18,s_19,s_20
0,1,1,-0.0005,0.0004,100.0,518.67,642.36,1583.23,1396.84,14.62,...,522.31,2388.01,8145.32,8.4246,0.03,391,2388,100.0,39.11,23.3537
1,1,2,0.0008,-0.0003,100.0,518.67,642.50,1584.69,1396.89,14.62,...,522.42,2388.03,8152.85,8.4403,0.03,392,2388,100.0,38.99,23.4491
2,1,3,-0.0014,-0.0002,100.0,518.67,642.18,1582.35,1405.61,14.62,...,522.03,2388.00,8150.17,8.3901,0.03,391,2388,100.0,38.85,23.3669
3,1,4,-0.0020,0.0001,100.0,518.67,642.92,1585.61,1392.27,14.62,...,522.49,2388.08,8146.56,8.3878,0.03,392,2388,100.0,38.96,23.2951
4,1,5,0.0016,0.0000,100.0,518.67,641.68,1588.63,1397.65,14.62,...,522.58,2388.03,8147.80,8.3869,0.03,392,2388,100.0,39.14,23.4583
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24715,100,148,-0.0016,-0.0003,100.0,518.67,643.78,1596.01,1424.11,14.62,...,519.66,2388.30,8138.08,8.5036,0.03,394,2388,100.0,38.44,22.9631
24716,100,149,0.0034,-0.0003,100.0,518.67,643.29,1596.38,1429.14,14.62,...,519.91,2388.28,8144.36,8.5174,0.03,395,2388,100.0,38.50,22.9746
24717,100,150,-0.0016,0.0004,100.0,518.67,643.84,1604.53,1431.41,14.62,...,519.44,2388.24,8135.95,8.5223,0.03,396,2388,100.0,38.39,23.0682
24718,100,151,-0.0023,0.0004,100.0,518.67,643.94,1597.56,1426.57,14.62,...,520.01,2388.26,8141.24,8.5148,0.03,395,2388,100.0,38.31,23.0753


## Preprocessing

### Validation Set

In [12]:
from sklearn.model_selection import GroupShuffleSplit

def train_val_split(train):
    gss = GroupShuffleSplit(n_splits=1, train_size=0.80, random_state=42)  
    for idx_train, idx_val in gss.split(train,groups=train["unit_number"]):
        # print('train_split_engines', train.iloc[idx_train]['unit_number'].unique(), '\n')
        # print('validate_split_engines', train.iloc[idx_val]['unit_number'].unique(), '\n')

        df_train = train.iloc[idx_train].copy()
        df_val = train.iloc[idx_val].copy()

    return df_train, df_val

In [13]:
df_train, df_val = train_val_split(train)

In [14]:
X_t_train, y_t_train = proc.X_y_train_divide(df_train)
X_t_val, y_t_val = proc.X_y_train_divide(df_val)

### Test Set Transformation 
Test set has samples for all cycles, but has annotations only for last one

In [15]:
test.shape, y_test.shape

((16596, 26), (100, 1))

In [16]:
test_last = proc.transform_test(test)
test_last.head()

,s_0,s_1,s_2,s_3,s_4,s_5,s_6,s_7,s_8,s_9,...,s_11,s_12,s_13,s_14,s_15,s_16,s_17,s_18,s_19,s_20
0,518.67,642.59,1592.40,1409.87,14.62,21.58,560.53,2388.22,9085.50,1.31,...,528.05,2388.23,8158.77,8.2966,0.03,393,2388,100.0,39.43,23.5679
1,518.67,642.56,1587.42,1409.69,14.62,21.61,553.33,2388.18,9050.97,1.30,...,520.90,2388.17,8128.04,8.4514,0.03,392,2388,100.0,38.83,23.2821
2,518.67,642.75,1591.93,1417.66,14.62,21.60,563.61,2388.31,9091.69,1.31,...,531.36,2388.33,8173.56,8.3057,0.03,395,2388,100.0,39.27,23.6440
3,518.67,642.28,1584.68,1406.56,14.62,21.61,552.75,2388.07,9048.23,1.30,...,521.27,2388.09,8133.78,8.4337,0.03,392,2388,100.0,38.70,23.3804
4,518.67,642.15,1580.59,1397.26,14.62,21.58,553.82,2387.96,9050.89,1.30,...,521.74,2387.96,8132.51,8.3900,0.03,390,2388,100.0,38.89,23.4463


In [17]:
X_test = test_last

### Remaining Useful Life (RUL)

In [18]:
train = proc.add_remaining_useful_life_linear(train)
train[index_cols+['RUL']].head()

,unit_number,time,RUL
0,1,1,258
1,1,2,257
2,1,3,256
3,1,4,255
4,1,5,254


## Attributes and target separation

In [19]:
X_train, y_train = proc.X_y_train_divide(train)

In [20]:
y_train.head()

,RUL
0,258
1,257
2,256
3,255
4,254


In [21]:
X_train.head()

,s_0,s_1,s_2,s_3,s_4,s_5,s_6,s_7,s_8,s_9,...,s_11,s_12,s_13,s_14,s_15,s_16,s_17,s_18,s_19,s_20
0,518.67,642.36,1583.23,1396.84,14.62,21.61,553.97,2387.96,9062.17,1.3,...,522.31,2388.01,8145.32,8.4246,0.03,391,2388,100.0,39.11,23.3537
1,518.67,642.50,1584.69,1396.89,14.62,21.61,554.55,2388.00,9061.78,1.3,...,522.42,2388.03,8152.85,8.4403,0.03,392,2388,100.0,38.99,23.4491
2,518.67,642.18,1582.35,1405.61,14.62,21.61,554.43,2388.03,9070.23,1.3,...,522.03,2388.00,8150.17,8.3901,0.03,391,2388,100.0,38.85,23.3669
3,518.67,642.92,1585.61,1392.27,14.62,21.61,555.21,2388.00,9064.57,1.3,...,522.49,2388.08,8146.56,8.3878,0.03,392,2388,100.0,38.96,23.2951
4,518.67,641.68,1588.63,1397.65,14.62,21.61,554.74,2388.04,9076.14,1.3,...,522.58,2388.03,8147.80,8.3869,0.03,392,2388,100.0,39.14,23.4583


## Training and Evaluation functions

In [22]:
eval = Evaluation()

In [23]:
search = HyperparameterSearch()

# MLP Construction

## Callbacks

In [24]:
# Early Stopping Callback
es = tf.keras.callbacks.EarlyStopping(monitor='loss', 
                                      patience=5, restore_best_weights=True)

In [25]:
# Printing Callback
def printLog(epoch, logs):
    print(
        f"E {epoch+1}\t: loss={logs['loss']:.3f}, "+
        f"rmse={logs['root_mean_squared_error']:.3f}, "+
        f"r2={logs['r_square']:.3f}; "+
        f"v_loss={logs['val_loss']:.3f}, "+
        f"v_rmse={logs['val_root_mean_squared_error']:.3f}, "+
        f"v_r2={logs['val_r_square']:.3f}; "
    )

printerCallback = LambdaCallback(on_epoch_end=printLog)

## Auxiliary HyperParameters

In [26]:
layer_sizes=[16,32,64,128,256,512]

In [27]:
# Exponential Decay Schedules
ED1 = ExponentialDecay(initial_learning_rate=1e-2, name="ED1",
                       decay_steps=100000, decay_rate=0.96)
ED2 = ExponentialDecay(initial_learning_rate=1e-2, name="ED2",
                       decay_steps=100000, decay_rate=0.8)
ED3 = ExponentialDecay(initial_learning_rate=1e-1, name="ED3",
                       decay_steps=100000, decay_rate=0.96)

In [28]:
# Generation of list combinations
from itertools import chain, permutations

def all_permutations(lst, size):
    result = list(chain.from_iterable([permutations(lst, x) for x in range(len(lst)+1)]))
    out = []
    for r in result:
        if (len(r) == size):
            out.append(list(r))
    return out

## Constants

In [29]:
# X_train must include indices
train3 = train.copy()
X_train_ = train3.drop(columns=[index_cols[1]]+["RUL"])

In [30]:
INPUT_SHAPE = 0

## Wrapper

In [31]:
from sys import is_finalizing
from sklearn.metrics import r2_score

class MLPWrapperRegressor(BaseEstimator,RegressorMixin):
    def __init__(self, basemodel=None, clip_y=-1,
                 include_settings=False, poly_degree=1,
                 scaler=StandardScaler()):
        # Base parameters
        self.basemodel = basemodel
        self.clip_y = clip_y
        self.poly_degree = poly_degree
        self.include_settings = include_settings

        # Column indexers
        self.feature_cols = sensors_cols
        if(include_settings):
            self.feature_cols = settings_cols + self.feature_cols
        self.base_feature_cols = self.feature_cols

        # Scaler and PolyFeatures transformers
        self.scaler = scaler
        self.polyft = PolynomialFeatures(degree=self.poly_degree, 
                                         include_bias=False)

    def clean_cols(self,df):
        if("unit_number" in df.columns): del df["unit_number"]
        if("unit_number" in df.columns): del df["unit_number"]
        if("time" in df.columns): del df["time"]
        if((not self.include_settings)): 
            for col in settings_cols:
                if(col in df.columns): del df[col]
        return df

    def fit(self, X=None, y=None):
        # Drop irrelevant column
        data = pd.DataFrame(X).copy()
        data = self.clean_cols(data)
        
        # Apply polynomial features and add them to the dataframe
        transf = self.polyft.fit_transform(data)
        data = pd.DataFrame(transf, columns=
                              self.polyft.get_feature_names_out())
        self.feature_cols = list(self.polyft.get_feature_names_out())

        # Scale the data
        data = self.scaler.fit_transform(data)
        data = pd.DataFrame(data, 
                            columns=self.scaler.get_feature_names_out())
        X_train = data.copy()
        
        # Clip and transform labels
        data2 = pd.DataFrame(y, columns=["RUL"]).copy()
        if (self.clip_y > 0):
            data2["RUL"].clip(upper=self.clip_y, inplace=True)
        y_train = data2

        # Update input shape for future use
        global INPUT_SHAPE
        INPUT_SHAPE = X_train.shape[1]

        # Fit model
        self.basemodel.fit(X_train,y_train)
        return self
        
    def predict(self, X=None):
        # Perform transformation, if not done
        if (len(X.columns) != len(self.feature_cols)):
            X_train = self.transform_features(X)
        else:
            X_train = X
        return self.basemodel.predict(X_train)

    def transform_features(self, df):
        # Drop irrelevant column
        data = pd.DataFrame(df).copy()
        data = self.clean_cols(data)
        
        # Apply polynomial features and add them to the dataframe
        transf = self.polyft.transform(data)
        data = pd.DataFrame(transf, columns=
                              self.polyft.get_feature_names_out())
        # self.feature_cols = list(self.polyft.get_feature_names_out())

        # Scale the data
        data = self.scaler.transform(data)
        data = pd.DataFrame(data, 
                            columns=self.scaler.get_feature_names_out())
        return data

    def score(self, X, y, sample_weight=None):
        X_test = self.transform_features(X)

        # Clip and transform labels
        data2 = pd.DataFrame(y, columns=["RUL"]).copy()
        if (self.clip_y > 0):
            data2["RUL"].clip(upper=self.clip_y, inplace=True)
        y_test = data2

        # Predict on test data
        y_pred = self.predict(X_test)
        return r2_score(y_test, y_pred, sample_weight=sample_weight)

## Test Data

In [32]:
def scale_test(test,model):
    test2 = pd.DataFrame(test)
    test2 = model.clean_cols(test2)

    # Apply polynomial features
    transf = model.polyft.transform(test2)
    test2 = pd.DataFrame(transf, 
                         columns=model.polyft.get_feature_names_out())

    # Scale the data (with train data parameters)
    test2 = model.scaler.transform(test2)
    test2 = pd.DataFrame(test2, 
                         columns=model.polyft.get_feature_names_out())
    return test2

## Constructor

In [33]:
def create_model(optim=Adam, learning_rate=1e-3, 
                 layer1=32  , activation1="tanh"    , dropout1=0.1,
                 layer2=None, activation2="tanh"    , dropout2=0.1,
                 layer3=None, activation3="tanh"    , dropout3=0.1,
                 layer4=None, activation4="tanh"    , dropout4=0.1,
                 print_summary=False, loss='mean_squared_error',
                 metrics=[tf.keras.metrics.MeanSquaredError()]):
    model = Sequential()

    # Fully Connected Layer
    model.add(Dense(layer1, input_dim=INPUT_SHAPE, activation=activation1))
    model.add(Dropout(dropout1))

    # Additional hidden layers
    if(layer2 is not None):
        model.add(Dense(layer2, activation=activation2))
        model.add(Dropout(dropout2))
        if (layer3 is not None):
            model.add(Dense(layer3, activation=activation3))
            model.add(Dropout(dropout3))
            if (layer4 is not None):
                model.add(Dense(layer4, activation=activation4))
                model.add(Dropout(dropout4))

    # Output Layer
    model.add(Dense(1))

    model.compile(loss=loss, optimizer=optim(learning_rate=learning_rate), 
                  metrics=metrics)
    
    if(print_summary): model.summary()
    return model

# MLP-1 

## Linear RUL

Score: 0.6216329789186076  
Test: 0.
```
('basemodel__batch_size', 140),
('basemodel__epochs', 40),
('basemodel__model__activation1', 'selu'),
('basemodel__model__dropout1', 0.6738055381845307),
('basemodel__model__layer1', 274),
('basemodel__model__learning_rate', 0.0026697581512638087),
('basemodel__model__optim',
keras.optimizers.optimizer_v2.adam.Adam),
('basemodel__validation_split', 0.1),
('scaler', StandardScaler())
```


In [34]:
CLIP=-1

model = MLPWrapperRegressor(
        clip_y=CLIP, scaler=StandardScaler(), poly_degree=1,
        basemodel=
            KerasRegressor(model=create_model,
                           batch_size=140,
                           epochs=40,
                           model__activation1='selu',
                           model__dropout1=0.6738055381845307, 
                           model__layer1=274, 
                           model__learning_rate=0.0026697581512638087,
                           model__optim=Adam,
                           validation_split=0.1, 
                           verbose=0, callbacks=[es, printerCallback],
                           model__metrics=[RMSE(), R2()],
                           model__loss='mse',
                           print_summary=True
                           )
    )
model

MLPWrapperRegressor(basemodel=KerasRegressor(batch_size=140, callbacks=[<keras.callbacks.EarlyStopping object at 0x00000174B0636520>, <keras.callbacks.LambdaCallback object at 0x00000174B06D61F0>], epochs=40, model=<function create_model at 0x00000174B0F1F820>, model__activation1='selu', model__dropout1=0.6738055381845307, model__layer1=274, model__learning_rate=0.0026697581512638087, model__loss='mse', model__metrics=[<keras.metrics.RootMeanSquaredError object at 0x00000174B06D62B0>, <tensorflow_addons.metrics.r_square.RSquare object at 0x00000174B06D6AC0>], model__optim=<class 'keras.optimizer_v2.adam.Adam'>, print_summary=True, validation_split=0.1, verbose=0))

In [35]:
model.fit(X_train_, y_train)
print("Finished:", datetime.datetime.now())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 274)               6028      
                                                                 
 dropout (Dropout)           (None, 274)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 275       
                                                                 
Total params: 6,303
Trainable params: 6,303
Non-trainable params: 0
_________________________________________________________________
E 1	: loss=15391.626, rmse=124.063, r2=-0.635; v_loss=13940.530, v_rmse=118.070, v_r2=-0.078; 
E 2	: loss=4297.826, rmse=65.558, r2=0.543; v_loss=7979.636, v_rmse=89.329, v_r2=0.383; 
E 3	: loss=3759.965, rmse=61.319, r2=0.601; v_loss=7625.051, v_rmse=87.322, v_r2=0.410; 
E 4	: loss=3654.921, rmse=60.456, r2=0.612; v

In [37]:
# Scaling and formatting test data
test_sc = scale_test(X_test,model)

# Clipping test labels
reclipped_y = y_test.copy()
if(CLIP > 0): reclipped_y = reclipped_y["RUL"].clip(upper=CLIP, inplace=False)

# Evaluation
eval.show_result(reclipped_y, model.basemodel.predict(test_sc))
print("Finished:", datetime.datetime.now())

R2=-0.477,RMSE=-50.311
Finished: 2022-11-02 09:35:06.085026


## Non-Linear RUL

Score: 0.8769850696112492  
Test: 0.
```
('basemodel__batch_size', 32),
('basemodel__epochs', 47),
('basemodel__model__activation1', 'tanh'),
('basemodel__model__dropout1', 0.1),
('basemodel__model__layer1', 512),
('basemodel__model__learning_rate', 0.004394643527896066),
('basemodel__model__optim',
keras.optimizers.optimizer_v2.adam.Adam),
('basemodel__validation_split', 0.1),
('clip_y', 80),
('scaler', MinMaxScaler())
```


In [38]:
CLIP=80

model = MLPWrapperRegressor(
        clip_y=CLIP, scaler=MinMaxScaler(), poly_degree=1,
        basemodel=
            KerasRegressor(model=create_model,
                           batch_size=32,
                           epochs=47,
                           model__activation1='tanh',
                           model__dropout1=0.1, 
                           model__layer1=512, 
                           model__learning_rate=0.004394643527896066,
                           model__optim=Adam,
                           validation_split=0.1, 
                           verbose=0, callbacks=[es, printerCallback],
                           model__metrics=[RMSE(), R2()],
                           model__loss='mse',
                           print_summary=True
                           )
    )
model

MLPWrapperRegressor(basemodel=KerasRegressor(batch_size=32, callbacks=[<keras.callbacks.EarlyStopping object at 0x00000174B0636520>, <keras.callbacks.LambdaCallback object at 0x00000174B06D61F0>], epochs=47, model=<function create_model at 0x00000174B0F1F820>, model__activation1='tanh', model__dropout1=0.1, model__layer1=512, model__learning_rate=0.004394643527896066, model__loss='mse', model__metrics=[<keras.metrics.RootMeanSquaredError object at 0x00000174B2EFCA30>, <tensorflow_addons.metrics.r_square.RSquare object at 0x00000174B2765700>], model__optim=<class 'keras.optimizer_v2.adam.Adam'>, print_summary=True, validation_split=0.1, verbose=0),
                    clip_y=80, scaler=MinMaxScaler())

In [39]:
model.fit(X_train_, y_train)
print("Finished:", datetime.datetime.now())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 512)               11264     
                                                                 
 dropout_1 (Dropout)         (None, 512)               0         
                                                                 
 dense_3 (Dense)             (None, 1)                 513       
                                                                 
Total params: 11,777
Trainable params: 11,777
Non-trainable params: 0
_________________________________________________________________
E 1	: loss=399.719, rmse=19.993, r2=0.242; v_loss=120.266, v_rmse=10.967, v_r2=0.788; 
E 2	: loss=90.087, rmse=9.491, r2=0.829; v_loss=110.534, v_rmse=10.514, v_r2=0.805; 
E 3	: loss=86.128, rmse=9.281, r2=0.837; v_loss=104.702, v_rmse=10.232, v_r2=0.815; 
E 4	: loss=83.285, rmse=9.126, r2=0.842; v_loss=99.042, v

In [40]:
# Scaling and formatting test data
test_sc = scale_test(X_test,model)

# Clipping test labels
reclipped_y = y_test.copy()
if(CLIP > 0): reclipped_y = reclipped_y["RUL"].clip(upper=CLIP, inplace=False)

# Evaluation
eval.show_result(reclipped_y, model.basemodel.predict(test_sc))
print("Finished:", datetime.datetime.now())

R2=0.845,RMSE=-9.917
Finished: 2022-11-02 09:37:59.851027


## PolyFeatures + Linear RUL 


Score: 0.623044428465008  
Test: 0.
```
('basemodel__batch_size', 114),
('basemodel__epochs', 50),
('basemodel__model__activation1', 'tanh'),
('basemodel__model__dropout1', 0.8089278925268311),
('basemodel__model__layer1', 453),
('basemodel__model__learning_rate', 0.00336601478011081),
('basemodel__model__optim',
keras.optimizers.optimizer_v2.adam.Adam),
('basemodel__validation_split', 0.11337798870650495),
('poly_degree', 3),
('scaler', StandardScaler())
```


In [47]:
CLIP=-1

model = MLPWrapperRegressor(
        clip_y=CLIP, scaler=StandardScaler(), poly_degree=3,
        basemodel=
            KerasRegressor(model=create_model,
                           batch_size=114,
                           epochs=50,
                           model__activation1='tanh',
                           model__dropout1=0.8089278925268311, 
                           model__layer1=453, 
                           model__learning_rate=0.00336601478011081,
                           model__optim=Adam,
                           validation_split=0.11337798870650495, 
                           verbose=0, callbacks=[es, printerCallback],
                           model__metrics=[RMSE(), R2()],
                           model__loss='mse',
                           print_summary=True
                           )
    )
model

MLPWrapperRegressor(basemodel=KerasRegressor(batch_size=114, callbacks=[<keras.callbacks.EarlyStopping object at 0x00000174B0636520>, <keras.callbacks.LambdaCallback object at 0x00000174B06D61F0>], epochs=50, model=<function create_model at 0x00000174B0F1F820>, model__activation1='tanh', model__dropout1=0.8089278925268311, model__layer1=453, model__learning_rate=0.00336601478011081, model__loss='mse', model__metrics=[<keras.metrics.RootMeanSquaredError object at 0x00000174C6BCA760>, <tensorflow_addons.metrics.r_square.RSquare object at 0x00000174C6BB5CD0>], model__optim=<class 'keras.optimizer_v2.adam.Adam'>, print_summary=True, validation_split=0.11337798870650495, verbose=0),
                    poly_degree=3)

In [48]:
model.fit(X_train_, y_train)
print("Finished:", datetime.datetime.now())

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 453)               916872    
                                                                 
 dropout_4 (Dropout)         (None, 453)               0         
                                                                 
 dense_9 (Dense)             (None, 1)                 454       
                                                                 
Total params: 917,326
Trainable params: 917,326
Non-trainable params: 0
_________________________________________________________________
E 1	: loss=13080.769, rmse=114.371, r2=-0.377; v_loss=14543.778, v_rmse=120.598, v_r2=-0.225; 
E 2	: loss=8968.767, rmse=94.704, r2=0.056; v_loss=13139.883, v_rmse=114.629, v_r2=-0.107; 
E 3	: loss=8105.598, rmse=90.031, r2=0.147; v_loss=11989.110, v_rmse=109.495, v_r2=-0.010; 
E 4	: loss=7624.253, rmse=87.317,

In [49]:
# Scaling and formatting test data
test_sc = scale_test(X_test,model)

# Clipping test labels
reclipped_y = y_test.copy()
if(CLIP > 0): reclipped_y = reclipped_y["RUL"].clip(upper=CLIP, inplace=False)

# Evaluation
eval.show_result(reclipped_y, model.basemodel.predict(test_sc))
print("Finished:", datetime.datetime.now())

R2=-0.375,RMSE=-48.532
Finished: 2022-11-02 09:43:53.407527


## PolyFeatures + Non-Linear RUL


Score: 0.8488892610573798  
Test: 0.
```
('basemodel__batch_size', 97),
('basemodel__epochs', 24),
('basemodel__model__activation1', 'relu'),
('basemodel__model__dropout1', 0.15096258494167586),
('basemodel__model__layer1', 264),
('basemodel__model__learning_rate', 0.0040754761072153145),
('basemodel__model__optim',
keras.optimizers.optimizer_v2.adam.Adam),
('basemodel__validation_split', 0.17873567459530804),
('clip_y', 105),
('poly_degree', 3),
('scaler', StandardScaler())
```

In [56]:
CLIP=105

model = MLPWrapperRegressor(
        clip_y=CLIP, scaler=StandardScaler(), poly_degree=3,
        basemodel=
            KerasRegressor(model=create_model,
                           batch_size=97,
                           epochs=250,
                           model__activation1='relu',
                           model__dropout1=0.15096258494167586, 
                           model__layer1=264, 
                           model__learning_rate=0.0040754761072153145,
                           model__optim=Adam,
                           validation_split=0.17873567459530804, 
                           verbose=0, callbacks=[es, printerCallback],
                           model__metrics=[RMSE(), R2()],
                           model__loss='mse',
                           print_summary=True
                           )
    )
model

MLPWrapperRegressor(basemodel=KerasRegressor(batch_size=97, callbacks=[<keras.callbacks.EarlyStopping object at 0x00000174B0636520>, <keras.callbacks.LambdaCallback object at 0x00000174B06D61F0>], epochs=250, model=<function create_model at 0x00000174B0F1F820>, model__activation1='relu', model__dropout1=0.15096258494167586, model__layer1=264, model__learning_rate=0.0040754761072153145, model__loss='mse', model__metrics=[<keras.metrics.RootMeanSquaredError object at 0x00000174FBCED2E0>, <tensorflow_addons.metrics.r_square.RSquare object at 0x00000174C54771F0>], model__optim=<class 'keras.optimizer_v2.adam.Adam'>, print_summary=True, validation_split=0.17873567459530804, verbose=0),
                    clip_y=105, poly_degree=3)

In [57]:
model.fit(X_train_, y_train)
print("Finished:", datetime.datetime.now())

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_14 (Dense)            (None, 264)               534336    
                                                                 
 dropout_7 (Dropout)         (None, 264)               0         
                                                                 
 dense_15 (Dense)            (None, 1)                 265       
                                                                 
Total params: 534,601
Trainable params: 534,601
Non-trainable params: 0
_________________________________________________________________
E 1	: loss=707.313, rmse=26.595, r2=0.338; v_loss=413.919, v_rmse=20.345, v_r2=0.627; 
E 2	: loss=369.791, rmse=19.230, r2=0.654; v_loss=375.372, v_rmse=19.375, v_r2=0.662; 
E 3	: loss=332.397, rmse=18.232, r2=0.689; v_loss=368.473, v_rmse=19.196, v_r2=0.668; 
E 4	: loss=325.942, rmse=18.054, r2=0.695; v_loss=2

In [58]:
# Scaling and formatting test data
test_sc = scale_test(X_test,model)

# Clipping test labels
reclipped_y = y_test.copy()
if(CLIP > 0): reclipped_y = reclipped_y["RUL"].clip(upper=CLIP, inplace=False)

# Evaluation
eval.show_result(reclipped_y, model.basemodel.predict(test_sc))
print("Finished:", datetime.datetime.now())

R2=0.814,RMSE=-14.569
Finished: 2022-11-02 09:53:36.695682


# MLP-2

## Linear RUL


Score: 0.6117982346232361  
Test: 0.
```
('basemodel__batch_size', 215),
('basemodel__epochs', 31),
('basemodel__model__activation1', 'sigmoid'),
('basemodel__model__activation2', 'elu'),
('basemodel__model__dropout1', 0.535540507143301),
('basemodel__model__dropout2', 0.2838929138188482),
('basemodel__model__layer1', 62),
('basemodel__model__layer2', 401),
('basemodel__model__learning_rate', 0.002945657435413331),
('basemodel__model__optim',
keras.optimizers.optimizer_v2.rmsprop.RMSprop),
('basemodel__validation_split', 0.3558423584914324),
('scaler', StandardScaler())
```

In [60]:
CLIP=-1

model = MLPWrapperRegressor(
        clip_y=CLIP, scaler=StandardScaler(), poly_degree=1,
        basemodel=
            KerasRegressor(model=create_model,
                           batch_size=215,
                           epochs=31,
                           model__activation1='sigmoid',
                           model__activation2='elu',
                           model__dropout1=0.535540507143301, 
                           model__dropout2=0.2838929138188482, 
                           model__layer1=62, 
                           model__layer2=401, 
                           model__learning_rate=0.002945657435413331,
                           model__optim=RMSprop,
                           validation_split=0.3558423584914324, 
                           verbose=0, callbacks=[es, printerCallback],
                           model__metrics=[RMSE(), R2()],
                           model__loss='mse',
                           print_summary=True
                           )
    )
model

MLPWrapperRegressor(basemodel=KerasRegressor(batch_size=215, callbacks=[<keras.callbacks.EarlyStopping object at 0x00000174B0636520>, <keras.callbacks.LambdaCallback object at 0x00000174B06D61F0>], epochs=31, model=<function create_model at 0x00000174B0F1F820>, model__activation1='sigmoid', model__activation2='elu', model__dropout1=0.535540507143301, model__dropout...del__layer2=401, model__learning_rate=0.002945657435413331, model__loss='mse', model__metrics=[<keras.metrics.RootMeanSquaredError object at 0x00000175F2634640>, <tensorflow_addons.metrics.r_square.RSquare object at 0x00000175F26347F0>], model__optim=<class 'keras.optimizer_v2.rmsprop.RMSprop'>, print_summary=True, validation_split=0.3558423584914324, verbose=0))

In [61]:
model.fit(X_train_, y_train)
print("Finished:", datetime.datetime.now())

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_16 (Dense)            (None, 62)                1364      
                                                                 
 dropout_8 (Dropout)         (None, 62)                0         
                                                                 
 dense_17 (Dense)            (None, 401)               25263     
                                                                 
 dropout_9 (Dropout)         (None, 401)               0         
                                                                 
 dense_18 (Dense)            (None, 1)                 402       
                                                                 
Total params: 27,029
Trainable params: 27,029
Non-trainable params: 0
_________________________________________________________________
E 1	: loss=8377.563, rmse=91.529, r2=0.194; v_loss

In [62]:
# Scaling and formatting test data
test_sc = scale_test(X_test,model)

# Clipping test labels
reclipped_y = y_test.copy()
if(CLIP > 0): reclipped_y = reclipped_y["RUL"].clip(upper=CLIP, inplace=False)

# Evaluation
eval.show_result(reclipped_y, model.basemodel.predict(test_sc))
print("Finished:", datetime.datetime.now())

R2=-1.014,RMSE=-58.744
Finished: 2022-11-02 09:58:00.855702


## Non-Linear RUL

Score: 0.8675039655819224  
Test: 0.
```
('basemodel__batch_size', 164),
('basemodel__epochs', 29),
('basemodel__model__activation1', 'elu'),
('basemodel__model__activation2', 'tanh'),
('basemodel__model__dropout1', 0.8791215719915316),
('basemodel__model__dropout2', 0.1),
('basemodel__model__layer1', 511),
('basemodel__model__layer2', 512),
('basemodel__model__learning_rate', 0.009861716088609234),
('basemodel__model__optim',
keras.optimizers.optimizer_v2.adam.Adam),
('basemodel__validation_split', 0.1),
('clip_y', 80),
('scaler', StandardScaler())
```

In [63]:
CLIP=80

model = MLPWrapperRegressor(
        clip_y=CLIP, scaler=StandardScaler(), poly_degree=1,
        basemodel=
            KerasRegressor(model=create_model,
                           batch_size=164,
                           epochs=29,
                           model__activation1='elu',
                           model__activation2='tanh',
                           model__dropout1=0.8791215719915316, 
                           model__dropout2=0.1, 
                           model__layer1=511, 
                           model__layer2=512, 
                           model__learning_rate=0.009861716088609234,
                           model__optim=Adam,
                           validation_split=0.1, 
                           verbose=0, callbacks=[es, printerCallback],
                           model__metrics=[RMSE(), R2()],
                           model__loss='mse',
                           print_summary=True
                           )
    )
model

MLPWrapperRegressor(basemodel=KerasRegressor(batch_size=164, callbacks=[<keras.callbacks.EarlyStopping object at 0x00000174B0636520>, <keras.callbacks.LambdaCallback object at 0x00000174B06D61F0>], epochs=29, model=<function create_model at 0x00000174B0F1F820>, model__activation1='elu', model__activation2='tanh', model__dropout1=0.8791215719915316, model__dropout2=...yer1=511, model__layer2=512, model__learning_rate=0.009861716088609234, model__loss='mse', model__metrics=[<keras.metrics.RootMeanSquaredError object at 0x0000017612D7D520>, <tensorflow_addons.metrics.r_square.RSquare object at 0x0000017616D885B0>], model__optim=<class 'keras.optimizer_v2.adam.Adam'>, print_summary=True, validation_split=0.1, verbose=0),
                    clip_y=80)

In [64]:
model.fit(X_train_, y_train)
print("Finished:", datetime.datetime.now())

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_19 (Dense)            (None, 511)               11242     
                                                                 
 dropout_10 (Dropout)        (None, 511)               0         
                                                                 
 dense_20 (Dense)            (None, 512)               262144    
                                                                 
 dropout_11 (Dropout)        (None, 512)               0         
                                                                 
 dense_21 (Dense)            (None, 1)                 513       
                                                                 
Total params: 273,899
Trainable params: 273,899
Non-trainable params: 0
_________________________________________________________________
E 1	: loss=438.358, rmse=20.937, r2=0.169; v_los

In [66]:
# Scaling and formatting test data
test_sc = scale_test(X_test,model)

# Clipping test labels
reclipped_y = y_test.copy()
if(CLIP > 0): reclipped_y = reclipped_y["RUL"].clip(upper=CLIP, inplace=False)

# Evaluation
eval.show_result(reclipped_y, model.basemodel.predict(test_sc))
print("Finished:", datetime.datetime.now())

R2=0.777,RMSE=-11.887
Finished: 2022-11-02 10:00:52.972008


## PolyFeatures + Linear RUL

Score: 0.6119811339203444  
Test: 0.
```
('basemodel__batch_size', 477),
('basemodel__epochs', 49),
('basemodel__model__activation1', 'relu'),
('basemodel__model__activation2', 'selu'),
('basemodel__model__dropout1', 0.18510920905532013),
('basemodel__model__dropout2', 0.2129692928941368),
('basemodel__model__layer1', 420),
('basemodel__model__layer2', 241),
('basemodel__model__learning_rate', 0.009500210759954515),
('basemodel__model__optim',
keras.optimizers.optimizer_v2.adam.Adam),
('basemodel__validation_split', 0.11902117833628473),
('poly_degree', 3),
('scaler', MinMaxScaler())
```



In [67]:
CLIP=-1

model = MLPWrapperRegressor(
        clip_y=CLIP, scaler=MinMaxScaler(), poly_degree=3,
        basemodel=
            KerasRegressor(model=create_model,
                           batch_size=477,
                           epochs=49,
                           model__activation1='relu',
                           model__activation2='selu',
                           model__dropout1=0.18510920905532013, 
                           model__dropout2=0.2129692928941368, 
                           model__layer1=420, 
                           model__layer2=241, 
                           model__learning_rate=0.009500210759954515,
                           model__optim=Adam,
                           validation_split=0.11902117833628473, 
                           verbose=0, callbacks=[es, printerCallback],
                           model__metrics=[RMSE(), R2()],
                           model__loss='mse',
                           print_summary=True
                           )
    )
model

MLPWrapperRegressor(basemodel=KerasRegressor(batch_size=477, callbacks=[<keras.callbacks.EarlyStopping object at 0x00000174B0636520>, <keras.callbacks.LambdaCallback object at 0x00000174B06D61F0>], epochs=49, model=<function create_model at 0x00000174B0F1F820>, model__activation1='relu', model__activation2='selu', model__dropout1=0.18510920905532013, model__dropout..._rate=0.009500210759954515, model__loss='mse', model__metrics=[<keras.metrics.RootMeanSquaredError object at 0x000001761C7876A0>, <tensorflow_addons.metrics.r_square.RSquare object at 0x000001761C78E9A0>], model__optim=<class 'keras.optimizer_v2.adam.Adam'>, print_summary=True, validation_split=0.11902117833628473, verbose=0),
                    poly_degree=3, scaler=MinMaxScaler())

In [68]:
model.fit(X_train_, y_train)
print("Finished:", datetime.datetime.now())

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_22 (Dense)            (None, 420)               850080    
                                                                 
 dropout_12 (Dropout)        (None, 420)               0         
                                                                 
 dense_23 (Dense)            (None, 241)               101461    
                                                                 
 dropout_13 (Dropout)        (None, 241)               0         
                                                                 
 dense_24 (Dense)            (None, 1)                 242       
                                                                 
Total params: 951,783
Trainable params: 951,783
Non-trainable params: 0
_________________________________________________________________
E 1	: loss=7340.618, rmse=85.677, r2=0.229; v_l

In [69]:
# Scaling and formatting test data
test_sc = scale_test(X_test,model)

# Clipping test labels
reclipped_y = y_test.copy()
if(CLIP > 0): reclipped_y = reclipped_y["RUL"].clip(upper=CLIP, inplace=False)

# Evaluation
eval.show_result(reclipped_y, model.basemodel.predict(test_sc))
print("Finished:", datetime.datetime.now())

R2=-0.706,RMSE=-54.072
Finished: 2022-11-02 10:01:28.025427


## PolyFeatures + Non-Linear RUL


Score: 0.8797075116940171  
Test: 0.
```
('basemodel__batch_size', 32),
('basemodel__epochs', 50),
('basemodel__model__activation1', 'tanh'),
('basemodel__model__activation2', 'elu'),
('basemodel__model__dropout1', 0.1),
('basemodel__model__dropout2', 0.1),
('basemodel__model__layer1', 461),
('basemodel__model__layer2', 512),
('basemodel__model__learning_rate', 0.0001),
('basemodel__model__optim',
keras.optimizers.optimizer_v2.adam.Adam),
('basemodel__validation_split', 0.1),
('clip_y', 80),
('poly_degree', 3),
('scaler', StandardScaler())
```

In [70]:
CLIP=80

model = MLPWrapperRegressor(
        clip_y=CLIP, scaler=StandardScaler(), poly_degree=3,
        basemodel=
            KerasRegressor(model=create_model,
                           batch_size=32,
                           epochs=50,
                           model__activation1='tanh',
                           model__activation2='elu',
                           model__dropout1=0.1, 
                           model__dropout2=0.1, 
                           model__layer1=461, 
                           model__layer2=512, 
                           model__learning_rate=0.0001,
                           model__optim=Adam,
                           validation_split=0.1, 
                           verbose=0, callbacks=[es, printerCallback],
                           model__metrics=[RMSE(), R2()],
                           model__loss='mse',
                           print_summary=True
                           )
    )
model

MLPWrapperRegressor(basemodel=KerasRegressor(batch_size=32, callbacks=[<keras.callbacks.EarlyStopping object at 0x00000174B0636520>, <keras.callbacks.LambdaCallback object at 0x00000174B06D61F0>], epochs=50, model=<function create_model at 0x00000174B0F1F820>, model__activation1='tanh', model__activation2='elu', model__dropout1=0.1, model__dropout2=0.1, model__layer1=461, model__layer2=512, model__learning_rate=0.0001, model__loss='mse', model__metrics=[<keras.metrics.RootMeanSquaredError object at 0x000001761CAFFD90>, <tensorflow_addons.metrics.r_square.RSquare object at 0x000001761CA680D0>], model__optim=<class 'keras.optimizer_v2.adam.Adam'>, print_summary=True, validation_split=0.1, verbose=0),
                    clip_y=80, poly_degree=3)

In [71]:
model.fit(X_train_, y_train)
print("Finished:", datetime.datetime.now())

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_25 (Dense)            (None, 461)               933064    
                                                                 
 dropout_14 (Dropout)        (None, 461)               0         
                                                                 
 dense_26 (Dense)            (None, 512)               236544    
                                                                 
 dropout_15 (Dropout)        (None, 512)               0         
                                                                 
 dense_27 (Dense)            (None, 1)                 513       
                                                                 
Total params: 1,170,121
Trainable params: 1,170,121
Non-trainable params: 0
_________________________________________________________________
E 1	: loss=695.343, rmse=26.369, r2=-0.318;

In [72]:
# Scaling and formatting test data
test_sc = scale_test(X_test,model)

# Clipping test labels
reclipped_y = y_test.copy()
if(CLIP > 0): reclipped_y = reclipped_y["RUL"].clip(upper=CLIP, inplace=False)

# Evaluation
eval.show_result(reclipped_y, model.basemodel.predict(test_sc))
print("Finished:", datetime.datetime.now())

R2=0.830,RMSE=-10.379
Finished: 2022-11-02 10:04:18.041178


# MLP-3

## Linear RUL

Score: 0.6031587839965488    
Test: 0.
```
('basemodel__batch_size', 273),
('basemodel__epochs', 18),
('basemodel__model__activation1', 'selu'),
('basemodel__model__activation2', 'tanh'),
('basemodel__model__activation3', 'selu'),
('basemodel__model__dropout1', 0.8779935839787812),
('basemodel__model__dropout2', 0.10779365305325281),
('basemodel__model__dropout3', 0.4510533904028945),
('basemodel__model__layer1', 244),
('basemodel__model__layer2', 341),
('basemodel__model__layer3', 104),
('basemodel__model__learning_rate', 0.006744014727126686),
('basemodel__model__optim',
keras.optimizers.optimizer_v2.rmsprop.RMSprop),
('basemodel__validation_split', 0.1),
('scaler', MinMaxScaler())
```


In [73]:
CLIP=-1

model = MLPWrapperRegressor(
        clip_y=CLIP, scaler=MinMaxScaler(), poly_degree=1,
        basemodel=
            KerasRegressor(model=create_model,
                           batch_size=273,
                           epochs=18,
                           model__activation1='selu',
                           model__activation2='tanh',
                           model__activation3='selu',
                           model__dropout1=0.8779935839787812, 
                           model__dropout2=0.10779365305325281, 
                           model__dropout3=0.4510533904028945, 
                           model__layer1=244, 
                           model__layer2=341, 
                           model__layer3=104, 
                           model__learning_rate=0.006744014727126686,
                           model__optim=RMSprop,
                           validation_split=0.1, 
                           verbose=0, callbacks=[es, printerCallback],
                           model__metrics=[RMSE(), R2()],
                           model__loss='mse',
                           print_summary=True
                           )
    )
model

MLPWrapperRegressor(basemodel=KerasRegressor(batch_size=273, callbacks=[<keras.callbacks.EarlyStopping object at 0x00000174B0636520>, <keras.callbacks.LambdaCallback object at 0x00000174B06D61F0>], epochs=18, model=<function create_model at 0x00000174B0F1F820>, model__activation1='selu', model__activation2='tanh', model__activation3='selu', model__dropout1=0.877993...yer3=104, model__learning_rate=0.006744014727126686, model__loss='mse', model__metrics=[<keras.metrics.RootMeanSquaredError object at 0x00000176151FA040>, <tensorflow_addons.metrics.r_square.RSquare object at 0x00000176151FA370>], model__optim=<class 'keras.optimizer_v2.rmsprop.RMSprop'>, print_summary=True, validation_split=0.1, verbose=0),
                    scaler=MinMaxScaler())

In [74]:
model.fit(X_train_, y_train)
print("Finished:", datetime.datetime.now())

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_28 (Dense)            (None, 244)               5368      
                                                                 
 dropout_16 (Dropout)        (None, 244)               0         
                                                                 
 dense_29 (Dense)            (None, 341)               83545     
                                                                 
 dropout_17 (Dropout)        (None, 341)               0         
                                                                 
 dense_30 (Dense)            (None, 104)               35568     
                                                                 
 dropout_18 (Dropout)        (None, 104)               0         
                                                                 
 dense_31 (Dense)            (None, 1)               

In [75]:
# Scaling and formatting test data
test_sc = scale_test(X_test,model)

# Clipping test labels
reclipped_y = y_test.copy()
if(CLIP > 0): reclipped_y = reclipped_y["RUL"].clip(upper=CLIP, inplace=False)

# Evaluation
eval.show_result(reclipped_y, model.basemodel.predict(test_sc))
print("Finished:", datetime.datetime.now())

R2=-0.641,RMSE=-53.034
Finished: 2022-11-02 10:07:33.222117


## Non-Linear RUL

Score: 0.8606739658341226    
Test: 0.
```
('basemodel__batch_size', 246),
('basemodel__epochs', 34),
('basemodel__model__activation1', 'relu'),
('basemodel__model__activation2', 'sigmoid'),
('basemodel__model__activation3', 'relu'),
('basemodel__model__dropout1', 0.398822951109552),
('basemodel__model__dropout2', 0.3447841740069549),
('basemodel__model__dropout3', 0.9),
('basemodel__model__layer1', 142),
('basemodel__model__layer2', 454),
('basemodel__model__layer3', 370),
('basemodel__model__learning_rate', 0.00337864526028697),
('basemodel__model__optim',
keras.optimizers.optimizer_v2.adam.Adam),
('basemodel__validation_split', 0.433392689037953),
('clip_y', 80),
('scaler', StandardScaler())
```


In [76]:
CLIP=80

model = MLPWrapperRegressor(
        clip_y=CLIP, scaler=StandardScaler(), poly_degree=1,
        basemodel=
            KerasRegressor(model=create_model,
                           batch_size=246,
                           epochs=34,
                           model__activation1='relu',
                           model__activation2='sigmoid',
                           model__activation3='relu',
                           model__dropout1=0.398822951109552, 
                           model__dropout2=0.3447841740069549, 
                           model__dropout3=0.9, 
                           model__layer1=142, 
                           model__layer2=454, 
                           model__layer3=370,  
                           model__learning_rate=0.00337864526028697,
                           model__optim=Adam,
                           validation_split=0.433392689037953, 
                           verbose=0, callbacks=[es, printerCallback],
                           model__metrics=[RMSE(), R2()],
                           model__loss='mse',
                           print_summary=True
                           )
    )
model

MLPWrapperRegressor(basemodel=KerasRegressor(batch_size=246, callbacks=[<keras.callbacks.EarlyStopping object at 0x00000174B0636520>, <keras.callbacks.LambdaCallback object at 0x00000174B06D61F0>], epochs=34, model=<function create_model at 0x00000174B0F1F820>, model__activation1='relu', model__activation2='sigmoid', model__activation3='relu', model__dropout1=0.398...l__layer3=370, model__learning_rate=0.00337864526028697, model__loss='mse', model__metrics=[<keras.metrics.RootMeanSquaredError object at 0x000001761CA43FD0>, <tensorflow_addons.metrics.r_square.RSquare object at 0x00000175F1C931F0>], model__optim=<class 'keras.optimizer_v2.adam.Adam'>, print_summary=True, validation_split=0.433392689037953, verbose=0),
                    clip_y=80)

In [77]:
model.fit(X_train_, y_train)
print("Finished:", datetime.datetime.now())

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_32 (Dense)            (None, 142)               3124      
                                                                 
 dropout_19 (Dropout)        (None, 142)               0         
                                                                 
 dense_33 (Dense)            (None, 454)               64922     
                                                                 
 dropout_20 (Dropout)        (None, 454)               0         
                                                                 
 dense_34 (Dense)            (None, 370)               168350    
                                                                 
 dropout_21 (Dropout)        (None, 370)               0         
                                                                 
 dense_35 (Dense)            (None, 1)               

In [78]:
# Scaling and formatting test data
test_sc = scale_test(X_test,model)

# Clipping test labels
reclipped_y = y_test.copy()
if(CLIP > 0): reclipped_y = reclipped_y["RUL"].clip(upper=CLIP, inplace=False)

# Evaluation
eval.show_result(reclipped_y, model.basemodel.predict(test_sc))
print("Finished:", datetime.datetime.now())

R2=0.829,RMSE=-10.424
Finished: 2022-11-02 10:09:58.140179


## PolyFeatures + Linear RUL

Score: 0.6144434017693393    
Test: 0.
```
('basemodel__batch_size', 392),
('basemodel__epochs', 25),
('basemodel__model__activation1', 'sigmoid'),
('basemodel__model__activation2', 'selu'),
('basemodel__model__activation3', 'selu'),
('basemodel__model__dropout1', 0.26670195337882074),
('basemodel__model__dropout2', 0.14342309787217927),
('basemodel__model__dropout3', 0.3763758002816012),
('basemodel__model__layer1', 211),
('basemodel__model__layer2', 16),
('basemodel__model__layer3', 398),
('basemodel__model__learning_rate', 0.005523882705085696),
('basemodel__model__optim',
keras.optimizers.optimizer_v2.adam.Adam),
('basemodel__validation_split', 0.1),
('poly_degree', 2),
('scaler', MinMaxScaler())
```


In [85]:
CLIP=-1

model = MLPWrapperRegressor(
        clip_y=CLIP, scaler=MinMaxScaler(), poly_degree=2,
        basemodel=
            KerasRegressor(model=create_model,
                           batch_size=392,
                           epochs=25,
                           model__activation1='sigmoid',
                           model__activation2='selu',
                           model__activation3='selu',
                           model__dropout1=0.26670195337882074, 
                           model__dropout2=0.14342309787217927, 
                           model__dropout3=0.3763758002816012, 
                           model__layer1=211, 
                           model__layer2=16, 
                           model__layer3=398, 
                           model__learning_rate=0.005523882705085696,
                           model__optim=Adam,
                           validation_split=0.1, 
                           verbose=0, callbacks=[es, printerCallback],
                           model__metrics=[RMSE(), R2()],
                           model__loss='mse',
                           print_summary=True
                           )
    )
model

MLPWrapperRegressor(basemodel=KerasRegressor(batch_size=392, callbacks=[<keras.callbacks.EarlyStopping object at 0x00000174B0636520>, <keras.callbacks.LambdaCallback object at 0x00000174B06D61F0>], epochs=25, model=<function create_model at 0x00000174B0F1F820>, model__activation1='sigmoid', model__activation2='selu', model__activation3='selu', model__dropout1=0.266..., model__learning_rate=0.005523882705085696, model__loss='mse', model__metrics=[<keras.metrics.RootMeanSquaredError object at 0x000001761D0D5CD0>, <tensorflow_addons.metrics.r_square.RSquare object at 0x000001761D0D5D30>], model__optim=<class 'keras.optimizer_v2.adam.Adam'>, print_summary=True, validation_split=0.1, verbose=0),
                    poly_degree=2, scaler=MinMaxScaler())

In [86]:
model.fit(X_train_, y_train)
print("Finished:", datetime.datetime.now())

Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_44 (Dense)            (None, 211)               53383     
                                                                 
 dropout_28 (Dropout)        (None, 211)               0         
                                                                 
 dense_45 (Dense)            (None, 16)                3392      
                                                                 
 dropout_29 (Dropout)        (None, 16)                0         
                                                                 
 dense_46 (Dense)            (None, 398)               6766      
                                                                 
 dropout_30 (Dropout)        (None, 398)               0         
                                                                 
 dense_47 (Dense)            (None, 1)               

In [87]:
# Scaling and formatting test data
test_sc = scale_test(X_test,model)

# Clipping test labels
reclipped_y = y_test.copy()
if(CLIP > 0): reclipped_y = reclipped_y["RUL"].clip(upper=CLIP, inplace=False)

# Evaluation
eval.show_result(reclipped_y, model.basemodel.predict(test_sc))
print("Finished:", datetime.datetime.now())

R2=-0.398,RMSE=-48.943
Finished: 2022-11-02 10:12:38.538711


## PolyFeatures + Non-Linear RUL


Score: 0.8700749435053768   
Test: 0.
```
('basemodel__batch_size', 403),
('basemodel__epochs', 30),
('basemodel__model__activation1', 'selu'),
('basemodel__model__activation2', 'selu'),
('basemodel__model__activation3', 'elu'),
('basemodel__model__dropout1', 0.4446452001317711),
('basemodel__model__dropout2', 0.6178728243917544),
('basemodel__model__dropout3', 0.48882961847253814),
('basemodel__model__layer1', 310),
('basemodel__model__layer2', 273),
('basemodel__model__layer3', 114),
('basemodel__model__learning_rate', 0.007023482013202204),
('basemodel__model__optim',
keras.optimizers.optimizer_v2.adam.Adam),
('basemodel__validation_split', 0.2747333733591927),
('clip_y', 90),
('poly_degree', 3),
('scaler', StandardScaler())
```

In [88]:
CLIP=90

model = MLPWrapperRegressor(
        clip_y=CLIP, scaler=StandardScaler(), poly_degree=3,
        basemodel=
            KerasRegressor(model=create_model,
                           batch_size=403,
                           epochs=30,
                           model__activation1='selu',
                           model__activation2='selu',
                           model__activation3='elu',
                           model__dropout1=0.4446452001317711, 
                           model__dropout2=0.6178728243917544, 
                           model__dropout3=0.48882961847253814, 
                           model__layer1=310, 
                           model__layer2=273, 
                           model__layer3=114, 
                           model__learning_rate=0.007023482013202204,
                           model__optim=Adam,
                           validation_split=0.2747333733591927, 
                           verbose=0, callbacks=[es, printerCallback],
                           model__metrics=[RMSE(), R2()],
                           model__loss='mse',
                           print_summary=True
                           )
    )
model

MLPWrapperRegressor(basemodel=KerasRegressor(batch_size=403, callbacks=[<keras.callbacks.EarlyStopping object at 0x00000174B0636520>, <keras.callbacks.LambdaCallback object at 0x00000174B06D61F0>], epochs=30, model=<function create_model at 0x00000174B0F1F820>, model__activation1='selu', model__activation2='selu', model__activation3='elu', model__dropout1=0.4446452...del__learning_rate=0.007023482013202204, model__loss='mse', model__metrics=[<keras.metrics.RootMeanSquaredError object at 0x000001761D328970>, <tensorflow_addons.metrics.r_square.RSquare object at 0x000001761D328B50>], model__optim=<class 'keras.optimizer_v2.adam.Adam'>, print_summary=True, validation_split=0.2747333733591927, verbose=0),
                    clip_y=90, poly_degree=3)

In [89]:
model.fit(X_train_, y_train)
print("Finished:", datetime.datetime.now())

Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_48 (Dense)            (None, 310)               627440    
                                                                 
 dropout_31 (Dropout)        (None, 310)               0         
                                                                 
 dense_49 (Dense)            (None, 273)               84903     
                                                                 
 dropout_32 (Dropout)        (None, 273)               0         
                                                                 
 dense_50 (Dense)            (None, 114)               31236     
                                                                 
 dropout_33 (Dropout)        (None, 114)               0         
                                                                 
 dense_51 (Dense)            (None, 1)               

In [90]:
# Scaling and formatting test data
test_sc = scale_test(X_test,model)

# Clipping test labels
reclipped_y = y_test.copy()
if(CLIP > 0): reclipped_y = reclipped_y["RUL"].clip(upper=CLIP, inplace=False)

# Evaluation
eval.show_result(reclipped_y, model.basemodel.predict(test_sc))
print("Finished:", datetime.datetime.now())

R2=0.825,RMSE=-12.086
Finished: 2022-11-02 10:13:11.271985


# MLP-4 (TODO)

## Linear RUL


Score: 0.5965333341401537    
Test: 0.
```
('basemodel__batch_size', 66),
('basemodel__epochs', 50),
('basemodel__model__activation1', 'selu'),
('basemodel__model__activation2', 'sigmoid'),
('basemodel__model__activation3', 'selu'),
('basemodel__model__activation4', 'relu'),
('basemodel__model__dropout1', 0.8385437522647914),
('basemodel__model__dropout2', 0.23198581914083677),
('basemodel__model__dropout3', 0.1),
('basemodel__model__dropout4', 0.18918890917273032),
('basemodel__model__layer1', 291),
('basemodel__model__layer2', 117),
('basemodel__model__layer3', 147),
('basemodel__model__layer4', 248),
('basemodel__model__learning_rate', 0.01),
('basemodel__model__optim',
keras.optimizers.optimizer_v2.rmsprop.RMSprop),
('basemodel__validation_split', 0.14404412076041206),
('scaler', MinMaxScaler())
```


In [91]:
CLIP=-1

model = MLPWrapperRegressor(
        clip_y=CLIP, scaler=MinMaxScaler(), poly_degree=1,
        basemodel=
            KerasRegressor(model=create_model,
                           batch_size=66,
                           epochs=50,
                           model__activation1='selu',
                           model__activation2='sigmoid',
                           model__activation3='selu',
                           model__activation4='relu',
                           model__dropout1=0.8385437522647914, 
                           model__dropout2=0.23198581914083677, 
                           model__dropout3=0.1, 
                           model__dropout4=0.18918890917273032, 
                           model__layer1=291, 
                           model__layer2=117, 
                           model__layer3=147, 
                           model__layer4=248, 
                           model__learning_rate=0.01,
                           model__optim=RMSprop,
                           validation_split=0.14404412076041206, 
                           verbose=0, callbacks=[es, printerCallback],
                           model__metrics=[RMSE(), R2()],
                           model__loss='mse',
                           print_summary=True
                           )
    )
model

MLPWrapperRegressor(basemodel=KerasRegressor(batch_size=66, callbacks=[<keras.callbacks.EarlyStopping object at 0x00000174B0636520>, <keras.callbacks.LambdaCallback object at 0x00000174B06D61F0>], epochs=50, model=<function create_model at 0x00000174B0F1F820>, model__activation1='selu', model__activation2='sigmoid', model__activation3='selu', model__activation4='re...yer4=248, model__learning_rate=0.01, model__loss='mse', model__metrics=[<keras.metrics.RootMeanSquaredError object at 0x000001761D7CBEB0>, <tensorflow_addons.metrics.r_square.RSquare object at 0x000001761D7CB550>], model__optim=<class 'keras.optimizer_v2.rmsprop.RMSprop'>, print_summary=True, validation_split=0.14404412076041206, verbose=0),
                    scaler=MinMaxScaler())

In [92]:
model.fit(X_train_, y_train)
print("Finished:", datetime.datetime.now())

Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_52 (Dense)            (None, 291)               6402      
                                                                 
 dropout_34 (Dropout)        (None, 291)               0         
                                                                 
 dense_53 (Dense)            (None, 117)               34164     
                                                                 
 dropout_35 (Dropout)        (None, 117)               0         
                                                                 
 dense_54 (Dense)            (None, 147)               17346     
                                                                 
 dropout_36 (Dropout)        (None, 147)               0         
                                                                 
 dense_55 (Dense)            (None, 248)             

In [93]:
# Scaling and formatting test data
test_sc = scale_test(X_test,model)

# Clipping test labels
reclipped_y = y_test.copy()
if(CLIP > 0): reclipped_y = reclipped_y["RUL"].clip(upper=CLIP, inplace=False)

# Evaluation
eval.show_result(reclipped_y, model.basemodel.predict(test_sc))
print("Finished:", datetime.datetime.now())

R2=-0.180,RMSE=-44.970
Finished: 2022-11-02 10:17:02.159267


## Non-Linear RUL

Score: 0.862360484973761    
Test: 0.
```
('basemodel__batch_size', 141),
('basemodel__epochs', 36),
('basemodel__model__activation1', 'selu'),
('basemodel__model__activation2', 'selu'),
('basemodel__model__activation3', 'sigmoid'),
('basemodel__model__activation4', 'selu'),
('basemodel__model__dropout1', 0.6030375513292561),
('basemodel__model__dropout2', 0.404789057124993),
('basemodel__model__dropout3', 0.6692037092668784),
('basemodel__model__dropout4', 0.5529044823633812),
('basemodel__model__layer1', 342),
('basemodel__model__layer2', 86),
('basemodel__model__layer3', 353),
('basemodel__model__layer4', 80),
('basemodel__model__learning_rate', 0.0035609790280109383),
('basemodel__model__optim',
keras.optimizers.optimizer_v2.adam.Adam),
('basemodel__validation_split', 0.12093724706982273),
('clip_y', 89),
('scaler', StandardScaler())
```


In [102]:
CLIP=89

model = MLPWrapperRegressor(
        clip_y=CLIP, scaler=StandardScaler(), poly_degree=1,
        basemodel=
            KerasRegressor(model=create_model,
                           batch_size=141,
                           epochs=36,
                           model__activation1='selu',
                           model__activation2='selu',
                           model__activation3='sigmoid',
                           model__activation4='selu',
                           model__dropout1=0.6030375513292561, 
                           model__dropout2=0.404789057124993, 
                           model__dropout3=0.6692037092668784, 
                           model__dropout4=0.5529044823633812, 
                           model__layer1=342, 
                           model__layer2=86, 
                           model__layer3=353, 
                           model__layer4=80, 
                           model__learning_rate=0.0035609790280109383,
                           model__optim=Adam,
                           validation_split=0.1, 
                           verbose=0, callbacks=[es, printerCallback],
                           model__metrics=[RMSE(), R2()],
                           model__loss='mse',
                           print_summary=True
                           )
    )
model

MLPWrapperRegressor(basemodel=KerasRegressor(batch_size=141, callbacks=[<keras.callbacks.EarlyStopping object at 0x00000174B0636520>, <keras.callbacks.LambdaCallback object at 0x00000174B06D61F0>], epochs=36, model=<function create_model at 0x00000174B0F1F820>, model__activation1='selu', model__activation2='selu', model__activation3='sigmoid', model__activation4='s...yer3=353, model__layer4=80, model__learning_rate=0.0035609790280109383, model__loss='mse', model__metrics=[<keras.metrics.RootMeanSquaredError object at 0x000001761EB15F70>, <tensorflow_addons.metrics.r_square.RSquare object at 0x0000017623D205E0>], model__optim=<class 'keras.optimizer_v2.adam.Adam'>, print_summary=True, validation_split=0.1, verbose=0),
                    clip_y=89)

In [103]:
model.fit(X_train_, y_train)
print("Finished:", datetime.datetime.now())

Model: "sequential_22"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_72 (Dense)            (None, 342)               7524      
                                                                 
 dropout_50 (Dropout)        (None, 342)               0         
                                                                 
 dense_73 (Dense)            (None, 86)                29498     
                                                                 
 dropout_51 (Dropout)        (None, 86)                0         
                                                                 
 dense_74 (Dense)            (None, 353)               30711     
                                                                 
 dropout_52 (Dropout)        (None, 353)               0         
                                                                 
 dense_75 (Dense)            (None, 80)              

In [104]:
# Scaling and formatting test data
test_sc = scale_test(X_test,model)

# Clipping test labels
reclipped_y = y_test.copy()
if(CLIP > 0): reclipped_y = reclipped_y["RUL"].clip(upper=CLIP, inplace=False)

# Evaluation
eval.show_result(reclipped_y, model.basemodel.predict(test_sc))
print("Finished:", datetime.datetime.now())

R2=0.817,RMSE=-12.221
Finished: 2022-11-02 10:23:27.433492


## PolyFeatures + Linear RUL

Score: 0.5965860921885896    
Test: 0.
```
('basemodel__batch_size', 297),
('basemodel__epochs', 4),
('basemodel__model__activation1', 'relu'),
('basemodel__model__activation2', 'elu'),
('basemodel__model__activation3', 'selu'),
('basemodel__model__activation4', 'relu'),
('basemodel__model__dropout1', 0.5444566159296994),
('basemodel__model__dropout2', 0.4338236298382706),
('basemodel__model__dropout3', 0.7048318485513446),
('basemodel__model__dropout4', 0.17180189230876136),
('basemodel__model__layer1', 458),
('basemodel__model__layer2', 113),
('basemodel__model__layer3', 161),
('basemodel__model__layer4', 494),
('basemodel__model__learning_rate', 0.0006019662806120351),
('basemodel__model__optim',
keras.optimizers.optimizer_v2.adam.Adam),
('basemodel__validation_split', 0.572593134337274),
('poly_degree', 3),
('scaler', StandardScaler())
```


In [116]:
CLIP=-1

model = MLPWrapperRegressor(
        clip_y=CLIP, scaler=StandardScaler(), poly_degree=3,
        basemodel=
            KerasRegressor(model=create_model,
                           batch_size=297,
                           epochs=4,
                           model__activation1='relu',
                           model__activation2='elu',
                           model__activation3='selu',
                           model__activation4='relu',
                           model__dropout1=0.5444566159296994, 
                           model__dropout2=0.4338236298382706, 
                           model__dropout3=0.7048318485513446, 
                           model__dropout4=0.17180189230876136, 
                           model__layer1=458, 
                           model__layer2=113, 
                           model__layer3=161, 
                           model__layer4=494, 
                           model__learning_rate=0.0006019662806120351,
                           model__optim=Adam,
                           validation_split=0.572593134337274, 
                           verbose=0, callbacks=[es, printerCallback],
                           model__metrics=[RMSE(), R2()],
                           model__loss='mse',
                           print_summary=True
                           )
    )
model

MLPWrapperRegressor(basemodel=KerasRegressor(batch_size=297, callbacks=[<keras.callbacks.EarlyStopping object at 0x00000174B0636520>, <keras.callbacks.LambdaCallback object at 0x00000174B06D61F0>], epochs=4, model=<function create_model at 0x00000174B0F1F820>, model__activation1='relu', model__activation2='elu', model__activation3='selu', model__activation4='relu',...er4=494, model__learning_rate=0.0006019662806120351, model__loss='mse', model__metrics=[<keras.metrics.RootMeanSquaredError object at 0x00000176273D9130>, <tensorflow_addons.metrics.r_square.RSquare object at 0x000001761D887D90>], model__optim=<class 'keras.optimizer_v2.adam.Adam'>, print_summary=True, validation_split=0.572593134337274, verbose=0),
                    poly_degree=3)

In [117]:
model.fit(X_train_, y_train)
print("Finished:", datetime.datetime.now())

Model: "sequential_25"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_87 (Dense)            (None, 458)               926992    
                                                                 
 dropout_62 (Dropout)        (None, 458)               0         
                                                                 
 dense_88 (Dense)            (None, 113)               51867     
                                                                 
 dropout_63 (Dropout)        (None, 113)               0         
                                                                 
 dense_89 (Dense)            (None, 161)               18354     
                                                                 
 dropout_64 (Dropout)        (None, 161)               0         
                                                                 
 dense_90 (Dense)            (None, 494)             

In [118]:
# Scaling and formatting test data
test_sc = scale_test(X_test,model)

# Clipping test labels
reclipped_y = y_test.copy()
if(CLIP > 0): reclipped_y = reclipped_y["RUL"].clip(upper=CLIP, inplace=False)

# Evaluation
eval.show_result(reclipped_y, model.basemodel.predict(test_sc))
print("Finished:", datetime.datetime.now())

R2=0.266,RMSE=-35.461
Finished: 2022-11-02 10:26:26.793202


## PolyFeatures + Non-Linear RUL


Score: 0.8644256248735079  
Test: 0.
```
('basemodel__batch_size', 69),
             ('basemodel__epochs', 48),
             ('basemodel__model__activation1', 'relu'),
             ('basemodel__model__activation2', 'sigmoid'),
             ('basemodel__model__activation3', 'relu'),
             ('basemodel__model__activation4', 'selu'),
             ('basemodel__model__dropout1', 0.7861582092207701),
             ('basemodel__model__dropout2', 0.1651211488004366),
             ('basemodel__model__dropout3', 0.3806276727312806),
             ('basemodel__model__dropout4', 0.5407685563014424),
             ('basemodel__model__layer1', 335),
             ('basemodel__model__layer2', 352),
             ('basemodel__model__layer3', 87),
             ('basemodel__model__layer4', 110),
             ('basemodel__model__learning_rate', 0.0013462596203423387),
             ('basemodel__model__optim',
              keras.optimizers.optimizer_v2.rmsprop.RMSprop),
             ('basemodel__validation_split', 0.40880873684195296),
             ('clip_y', 84),
             ('poly_degree', 3),
             ('scaler', StandardScaler())
```

In [124]:
CLIP=84

model = MLPWrapperRegressor(
        clip_y=CLIP, scaler=StandardScaler(), poly_degree=3,
        basemodel=
            KerasRegressor(model=create_model,
                           batch_size=69,
                           epochs=48,
                           model__activation1='relu',
                           model__activation2='sigmoid',
                           model__activation3='relu',
                           model__activation4='selu',
                           model__dropout1=0.7861582092207701, 
                           model__dropout2=0.1651211488004366, 
                           model__dropout3=0.3806276727312806, 
                           model__dropout4=0.5407685563014424, 
                           model__layer1=335, 
                           model__layer2=352, 
                           model__layer3=87, 
                           model__layer4=110, 
                           model__learning_rate=0.0013462596203423387,
                           model__optim=RMSprop,
                           validation_split=0.40880873684195296, 
                           verbose=0, callbacks=[es, printerCallback],
                           model__metrics=[RMSE(), R2()],
                           model__loss='mse',
                           print_summary=True
                           )
    )
model

MLPWrapperRegressor(basemodel=KerasRegressor(batch_size=69, callbacks=[<keras.callbacks.EarlyStopping object at 0x00000174B0636520>, <keras.callbacks.LambdaCallback object at 0x00000174B06D61F0>], epochs=48, model=<function create_model at 0x00000174B0F1F820>, model__activation1='relu', model__activation2='sigmoid', model__activation3='relu', model__activation4='se...rning_rate=0.0013462596203423387, model__loss='mse', model__metrics=[<keras.metrics.RootMeanSquaredError object at 0x000001762782C790>, <tensorflow_addons.metrics.r_square.RSquare object at 0x0000017617095160>], model__optim=<class 'keras.optimizer_v2.rmsprop.RMSprop'>, print_summary=True, validation_split=0.40880873684195296, verbose=0),
                    clip_y=84, poly_degree=3)

In [125]:
model.fit(X_train_, y_train)
print("Finished:", datetime.datetime.now())

Model: "sequential_27"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_97 (Dense)            (None, 335)               678040    
                                                                 
 dropout_70 (Dropout)        (None, 335)               0         
                                                                 
 dense_98 (Dense)            (None, 352)               118272    
                                                                 
 dropout_71 (Dropout)        (None, 352)               0         
                                                                 
 dense_99 (Dense)            (None, 87)                30711     
                                                                 
 dropout_72 (Dropout)        (None, 87)                0         
                                                                 
 dense_100 (Dense)           (None, 110)             

In [127]:
# Scaling and formatting test data
test_sc = scale_test(X_test,model)

# Clipping test labels
reclipped_y = y_test.copy()
if(CLIP > 0): reclipped_y = reclipped_y["RUL"].clip(upper=CLIP, inplace=False)

# Evaluation
eval.show_result(reclipped_y, model.basemodel.predict(test_sc))
print("Finished:", datetime.datetime.now())

R2=0.316,RMSE=-22.091
Finished: 2022-11-02 10:32:43.843187
